In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [ ]:
import os

# 디버깅을 위한 프로젝트명을 기입합니다.
os.environ["LANGCHAIN_PROJECT"] = "RAG TUTORIAL"

# tracing 을 위해서는 아래 코드의 주석을 해제하고 실행합니다.
# os.environ["LANGCHAIN_TRACING_V2"] = true

In [ ]:
import bs4
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma, FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

In [ ]:
from langchain.document_loaders import PyPDFLoader

# PDF 파일 로드. 파일의 경로 입력
loader = PyPDFLoader("./reports/SPRI_AI_Brief_2023년12월호_F.pdf")

# 페이지 별 문서 로드
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[10].page_content[:500]}")
print(f"\n[metadata]\n{docs[10].metadata}\n")

In [2]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("./result/여B_의무기록.txt")
docs = loader.load()
print(f"문서의 수: {len(docs)}")

# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")

문서의 수: 1

[페이지내용]
환자 차트 요약:

* 작성일시: 2023년 6월 12일 14:37
* 진료과목: FM1
* 입원일시: 2023년 6월 12일 14:00
* 병실: 5병동 511호
* 성별: 여성
* 나이: 76세 (현재 나이: 77세)
* 활력 징후: 혈압: 116/85, 호흡수: 20, 체온: 36.8°C, 혈당: 261, 산소포화도: 98%
* 진단:
	+ 고지질혈증 (Unspecified hyperlipidemia, E785^00)
	+ 급성 세뇨관-간질신장염 (Acute tubulointerstitial nephritis, N10^00)
	+ 패혈증 (Unspecified sepsis, A419^00)
	+ 폐렴 (Unspecified pneumonia, J189^00)
	+ 심부정맥혈전증 (Deep vein thrombosis NOS, I802^01)
	+ 반코마이신 내성 (Vancomycin resistance, U830^00)
	+ 뇌경색증의 후유증 (Sequelae of cerebr

[metadata]
{'source': './result/여B_의무기록.txt'}



In [1]:
# !pip install unstructured

In [17]:
from langchain_community.document_loaders import DirectoryLoader

loader = DirectoryLoader("./results/남A_의무기록지/", glob="*.txt", show_progress=True)
docs = loader.load()

print(f"문서의 수: {len(docs)}")

100%|██████████| 452/452 [00:21<00:00, 21.49it/s] 

문서의 수: 452


In [19]:
# 10번째 페이지의 내용 출력
print(f"\n[페이지내용]\n{docs[0].page_content[:500]}")
print(f"\n[metadata]\n{docs[0].metadata}\n")


[페이지내용]
환자 정보:

환자 이름: 0092106935

성별: 남성

나이: 77세

병실: 312호

간호 기록:

1. 07:50 (날짜 미기재):

작성자: 병실 순회

환자 상태: 와상 상태, 왼쪽 편마비

L-튜브 (18Fr) 유지 및 경관식 장솔루션 400-400-400kcal/day 제공

대소변 조절 불가능, 기저귀 착용 중 (기저귀마다 약간의 변이 매달려 있음)

반복적인 행동: 오른손으로 침대 난간을 툭툭침

본원 에어매트 사용 중, 공기 누출 확인

체위 변경 및 등 마사지 시행

낙상 위험성에 대한 교육 제공 및 병상 두드리기와 같은 반복적인 행동에 대한 정서적 간호

1. 10:00 (날짜 미기재):

작성자: (V/S)

활력 징후: BP: 90/49, HR: 65, RESP: 20, Temp: 36.0°C

담당의 회진

1. 16:00 및 23:00 (날짜 미기재):

작성자: 병실 순회

환자 상태 관찰

침상 난간 고정 및 안정 유지

작성자: 병실 순회, 

[metadata]
{'source': 'results/남A_의무기록지/289.txt'}



In [20]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap  = 100,
    length_function = len,
)

texts = text_splitter.split_text(docs[0].page_content)

len(texts)

2

In [21]:
texts[0]

'환자 정보:\n\n환자 이름: 0092106935\n\n성별: 남성\n\n나이: 77세\n\n병실: 312호\n\n간호 기록:\n\n1. 07:50 (날짜 미기재):\n\n작성자: 병실 순회\n\n환자 상태: 와상 상태, 왼쪽 편마비\n\nL-튜브 (18Fr) 유지 및 경관식 장솔루션 400-400-400kcal/day 제공\n\n대소변 조절 불가능, 기저귀 착용 중 (기저귀마다 약간의 변이 매달려 있음)\n\n반복적인 행동: 오른손으로 침대 난간을 툭툭침\n\n본원 에어매트 사용 중, 공기 누출 확인\n\n체위 변경 및 등 마사지 시행\n\n낙상 위험성에 대한 교육 제공 및 병상 두드리기와 같은 반복적인 행동에 대한 정서적 간호\n\n1. 10:00 (날짜 미기재):\n\n작성자: (V/S)\n\n활력 징후: BP: 90/49, HR: 65, RESP: 20, Temp: 36.0°C\n\n담당의 회진\n\n1. 16:00 및 23:00 (날짜 미기재):\n\n작성자: 병실 순회\n\n환자 상태 관찰\n\n침상 난간 고정 및 안정 유지'

In [2]:
# !pip install chromadb

In [26]:
from dotenv import load_dotenv
import os
import json
import sys
import time
from tqdm import tqdm
# from langchain_community.vectorstores import Chroma
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_upstage import UpstageEmbeddings

# Load environment variables
load_dotenv()

# Load API keys from environment variables
UPSTAGE_API_KEY = os.getenv('UPSTAGE_API_KEY')

start_time = time.time()

# Use the recursive character splitter
recur_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=60,
    separators=["\n\n", "\n", "-", " ", ""]
)

# Perform the splits using the splitter
data_splits = list(tqdm(recur_splitter.split_documents(docs), desc="Splitting documents", unit="chunk"))
print(f"Number of splits: {len(data_splits)}")

# Vector Store 구축
embeddings = UpstageEmbeddings(
    api_key=UPSTAGE_API_KEY,
    model="solar-embedding-1-large"
)

persist_directory = '.cache/db/남A_의무기록지'

vectordb = Chroma.from_documents(
    documents=data_splits, # 위에서 처리한 데이터 
    embedding=embeddings, # upstage solar embedding 1 large
    persist_directory=persist_directory)

vectordb.persist()
vectordb = None

end_time = time.time()
elapsed_time = end_time - start_time

Splitting documents: 100%|██████████| 580/580 [00:00<00:00, 2479812.76chunk/s]


Number of splits: 580


In [45]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

from langchain_core.runnables import RunnablePassthrough

from langchain.vectorstores import Chroma
from langchain_upstage import ChatUpstage, UpstageEmbeddings


In [46]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE="""

# Your role
    - You are a compassionate, articulate physician.

------
    
# Instructions
    - Your goal is to explain medical information in a way that is easy for your patients to understand, avoiding complex medical jargon as much as possible.
    - Given a medical document or chart, it's your job to explain the key information that the patient or their family asks about in a patient-friendly format. 
    - When specific details are provided, such as diagnosis codes or medical history, simplify these terms and explain them in a way that is easy to understand.

------

# Document: \n\t{context}

------

# Question: \n\t{question}

------

# IMPORTANT 
    - Answer in KOREAN
    - Let us know the metadata to the document you referenced

# Answer :
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [47]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

가장 최근 검사 결과는 다음과 같습니다:

| 검사명 / 소견 | 결과치 | 참고치 |
| --- | --- | --- |
| AL | 5.00 ≤ 40 U/L |  |
| -GTP | 22.00 ≤ 64 U/L |  |
| Glucose | 139.00 H 60 ~ < 100 mg/dL |  |
| Total Bilirubin | 0.70 0.3 ~ 1.2 mg/dL |  |
| BUN | 28. 10 H 7 9 ~ 25.0 mg/dL |  |
| Creatinine | 0.80 0.77 ~ 1.43 mg/dL |  |
| A/G ratio | 1.20 Not established |  |
| eGFR | 92.00 > 59 mL/min/1.73m2 |  |

이 검사 결과에 대한 구체적인 설명이나 추가 정보가 필요하시면 의료진에게 문의해주세요.


In [48]:
print(qa_chain.invoke(query))

환자분, 2024년 5월 3일에 시행한 검사 결과에 대해 설명해 드리겠습니다. 이 검사는 간 기능, 혈당, 신장 기능 등을 평가하기 위한 것입니다.

간 기능을 나타내는 검사 결과로는 AL (알라닌 아미노전이효소)와 GTP (감마-글루타밀전달효소)가 있습니다. AL의 수치는 5.00 U/L 이하여야 하며, GTP의 수치는 22.00 U/L 이하여야 합니다. 그러나 검사 결과에서는 AL의 수치가 40 U/L을 초과하고, GTP의 수치가 64 U/L을 초과하였습니다. 이는 간 기능에 문제가 있을 수 있다는 것을 시사합니다.

혈당 수치를 나타내는 Glucose의 경우, 정상 범위는 60 ~ 100 mg/dL입니다. 그러나 검사 결과에서는 139.00 mg/dL로 높게 나타났습니다. 이는 당뇨병이나 기타 혈당 관련 문제가 있을 수 있다는 것을 시사합니다.

신장 기능을 나타내는 검사 결과로는 BUN (혈액요소질소)과 Creatinine이 있습니다. BUN의 수치는 7 9 ~ 25.0 mg/dL이며, Creatinine의 수치는 0.77 ~ 1.43 mg/dL입니다. 그러나 검사 결과에서는 BUN의 수치가 28.00 mg/dL로 높게 나타났습니다. 이는 신장 기능이 저하되어 노폐물이 체내에 쌓이고 있다는 것을 의미할 수 있습니다.

이러한 검사 결과를 종합해보면, 환자분의 간 기능과 신장 기능이 저하되어 있으며, 혈당 수치도 높은 것으로 나타났습니다. 이에 대한 추가적인 검사와 치료가 필요할 수 있습니다. 의료진과 상담하여 자세한 내용을 확인하시고, 치료 계획을 수립하시는 것이 좋습니다.


In [64]:
llm = ChatUpstage(api_key=os.getenv("UPSTAGE_API_KEY"))


# Embeddings setup
embeddings = UpstageEmbeddings(
  api_key=os.getenv("UPSTAGE_API_KEY"),
  model="solar-embedding-1-large"
)

vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embeddings
)

retriever = vectordb.as_retriever()

RAG_PROMPT_TEMPLATE='''
### 시스템 설명
당신은 병원 AI 지원 시스템입니다. 보호자 또는 환자가 의료 상태와 관련된 질문을 하면, 환자의 최신 의료 기록을 기반으로 적절한 답변을 제공해야 합니다.
보호자와 환자에게 알기 쉽게 친절하게 답변해주세요.
아래에 환자의 의료 기록이 제공됩니다.

### 의료 기록
{context}

### 질문
{question}

### 응답
환자의 상태, 식사, 약물 복용, 거동 상태 등을 포함하여 질문에 대한 정확한 답변을 생성하십시오.
참고한 데이터의 메타데이터를 알려주세요
'''

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT_TEMPLATE)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Define the RAG chain
qa_chain = (
    {
        "context": retriever | format_docs,
        "question": RunnablePassthrough(),
    }
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [65]:
query = "가장 최근 검사의 전반적인 결과가 어떻게 될까?"
print(qa_chain.invoke(query))

가장 최근 검사 결과(2024년 05월 03일)를 보면, 환자의 총 빌리루빈, -GTP, A/G ratio, eGFR, 알부민/글로불린 비율, 혈당, BUN, 크레아티닌 등이 정상 범위 내에 있습니다. 다만, 알부민 수치가 참고치(5.00 ≤ 40 U/L)를 벗어나 약간 높은 상태입니다.

검사 결과는 참고치와의 비교를 통해 환자의 건강 상태를 평가하게 됩니다.


In [66]:
print(qa_chain.invoke('말씀은 잘 하세요?'))

제공된 의료 기록에는 환자의 말하기 능력에 대한 정보가 없습니다. 따라서 주어진 정보로는 질문에 대한 정확한 답변을 제공할 수 없습니다.


In [67]:
print(qa_chain.invoke('밤에 잘 주무세요?'))

환자 간호 기록에 따르면, 환자는 밤 동안 특이 사항 없이 잘 잤습니다. 그러나 환자의 반복적인 행동 양상과 깨어 있을 때의 상태는 의료진의 관찰이 필요한 부분입니다. 환자의 간호 기록에는 밤 동안의 체위 변경, 등 마사지, 정서적인 간호 등이 포함되어 있습니다.


In [68]:
print(qa_chain.invoke('거동은 어떠세요?'))

환자님은 와상환자이며, 왼쪽 편마비가 있습니다.


In [69]:
print(qa_chain.invoke('약은 잘 드세요?'))

환자 차트 요약에 따르면, 환자는 약을 잘 복용하고 있습니다. 차트에는 환자가 2024년 5월 17일 10시 16분에 아니스펜 8시간 이알서방정(아세트아미노펜)을 복용했다고 기록되어 있습니다. 또한, 다른 약물들도 지정된 시간에 복용하고 있습니다.


In [70]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

환자는 상지 및 하지 마비, 욕창, 기침, 불안정한 혈압 등의 증상을 겪고 있습니다. 그러나 기록에는 환자가 얼마나 아픈지에 대한 직접적인 언급은 없습니다.

환자 차트의 메타데이터:

* 날짜: 2023년 10월 19일
* 부서: FM1
* 진단: 뇌내출혈, 수면장애, 위장염, 인후통, 폐렴, 편마비, 폐부종, 고혈압, 당뇨병, 고지혈증, 욕창, 항생제 내성, 세균 감염 등
* 치료 계획: 활력 징후 모니터링, 침상 안정, 자세 변경, 튜브 영양, 약물 투여 등
* 참고 사항: 진단 코드에 대한 심각도 수준이 제공됩니다.

간호 기록의 메타데이터:

* 날짜: 2023년 7월 5일
* 부서: 알 수 없음
* 진단: 알 수 없음
* 의료 기록: 알 수 없음
* 현재 약물: 알 수 없음
* 간호 기록: 환자의 상태, L-tube 유지, 욕창 간호, 바이탈 사인, 의사의 회진, 보호자와의 대화 등에 대한 기록이 포함되어 있습니다.


In [71]:
print(qa_chain.invoke('많이 아파하지는 않으신 가요?'))

환자의 활력 징후와 의료 기록을 보면, 환자는 현재 여러 가지 건강 문제로 인해 고통받고 있습니다. 특히, 뇌내출혈, 위장염, 폐렴, 당뇨병, 고혈압, 고지혈증 등의 진단을 받았습니다. 또한, 환자의 활력 징후가 불안정하게 나타나고 있습니다. 그러나, 환자가 얼마나 아픈지에 대한 구체적인 정보는 제공되지 않았습니다.


In [72]:
q = '아버님은 어떠세요?'
qa_chain.invoke(q)

'환자는 77세 남성으로, 반코마이신 내성, 카바페넴 내성, 기타 세균성 폐렴 등의 진단을 받았습니다. 현재 치료 계획은 침상 안정, 2시간마다 체위 변경, 경관식 (그린비아가정용장용액 1200Kcal/day) 등을 포함하고 있습니다. 또한, 트라젠타정, 네비스톨정, 비엘피린장용정 등 여러 약물을 복용하고 있습니다.'